In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.grid_search import GridSearchCV

/home/rohit_mattah/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/rohit_mattah/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
train = pd.io.parsers.read_csv("train.csv")

/home/rohit_mattah/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (294,299,300,302,307,313,315,365,367,369) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
test = pd.io.parsers.read_csv("test.csv")

/home/rohit_mattah/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (294,299,300,302,313,315,365,367,369) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
i1 = pd.DataFrame(test.UCIC_ID)

In [5]:
train = train.drop(['UCIC_ID','Percent_Change_in_Credits','Percent_Change_in_FT_Bank','Percent_Change_in_FT_outside','Percent_Change_in_Self_Txn','Percent_Change_in_Big_Expenses'], axis=1)

In [6]:
y = train.Responders.ravel()
train= train.drop('Responders',axis=1)

In [7]:
test = test.drop(['UCIC_ID','Percent_Change_in_Credits','Percent_Change_in_FT_Bank','Percent_Change_in_FT_outside','Percent_Change_in_Self_Txn','Percent_Change_in_Big_Expenses'], axis=1)

In [8]:
combined_set = pd.concat([train, test], axis = 0)

In [9]:
for feature in combined_set.columns: # Loop through all columns in the dataframe
    if combined_set[feature].dtype == 'object': # Only apply for columns with categ
        combined_set[feature] = pd.Categorical(combined_set[feature]).codes # Repla

In [10]:
train = combined_set.iloc[0:300000].values
test = combined_set.iloc[300000:].values

In [11]:
xgdmat = xgb.DMatrix(train, y)

In [12]:
test1 = xgb.DMatrix(test)

In [19]:
our_params = {'eta': 0.08, 'seed':0, 'subsample': 0.7, 'colsample_bytree': 0.6, 
             'objective': 'multi:softprob', 'max_depth':12, 'num_class' : 2, 'min_child_weight':1} 

In [20]:
cv_xgb = xgb.cv(params = our_params, dtrain = xgdmat, num_boost_round = 300, nfold =5 ,
                metrics = ['mlogloss'], 
                early_stopping_rounds = 20) 

In [21]:
cv_xgb.tail()

,test-mlogloss-mean,test-mlogloss-std,train-mlogloss-mean,train-mlogloss-std
81,0.303248,0.001611,0.153475,0.000636
82,0.303251,0.001618,0.152746,0.000634
83,0.303278,0.001609,0.152028,0.000650
84,0.303258,0.001600,0.151378,0.000509
85,0.303232,0.001623,0.150665,0.000493


In [22]:
final_gb = xgb.train(our_params, xgdmat, num_boost_round = 85)

In [23]:
pred = final_gb.predict(test1)

In [24]:
i2 = pred[:,1]
i1.columns = ['UCIC_ID']
i2 = pd.DataFrame(i2)
i2.columns = ['RESPONDERS']

In [25]:
result = pd.concat([i1, i2], axis=1)

In [26]:
result.head()

,UCIC_ID,RESPONDERS
0,337734,0.226470
1,488166,0.024952
2,410785,0.039802
3,389145,0.186978
4,221090,0.233672


In [27]:
result.to_csv('pred.csv', index= False)